In [0]:
# Python

# No external imports needed beyond stdlib allowed; iterates points directly (never points.points)
import re

def _extract_ticker_from_title(title):
    if not title: return None
    first = re.split(r"\||\-|–|—", title)[0].strip()
    cand = re.sub(r"[^A-Z0-9\.\-]", "", first.upper())
    return cand if 1 <= len(cand) <= 6 else None

def _extract_ticker_from_metadata(md):
    if not isinstance(md, dict): return None
    for k in ("Ticker","ticker","Symbol","symbol"):
        v = md.get(k)
        if isinstance(v, str) and v.strip():
            vv = re.sub(r"[^A-Z0-9\.\-]", "", v.strip().upper())
            if vv: return vv
    return None

def discover_tickers(points_obj, limit=None):
    out, seen = [], set()
    for p in points_obj:
        if limit and len(out) >= limit: break
        t = _extract_ticker_from_title(getattr(p, "title", None)) or _extract_ticker_from_metadata(getattr(p, "metadata", None))
        if t and t not in seen:
            seen.add(t); out.append(t)
    return out

try:
    tickers = discover_tickers(points)
    if not tickers:
        tickers = ["AAPL","MSFT","SPY","TSLA"]
except Exception as e:
    print (e)
    tickers = ["AAPL","MSFT","SPY","TSLA"]

# Mount the widget; TSX will fetch data itself
widget = TickerExplorer(tickers)
UI.addWidget(widget)
